In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Cape_July24.csv')


/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_87856/3061799828.py:1: DtypeWarning: Columns (93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Cape_July24.csv')


In [3]:
pd.set_option('display.max_columns', None)
dft = df[['Pitcher','PitcherTeam','RelHeight', 'SpinRate', 'RelSpeed','PitchCall','TaggedPitchType','SpinRate','HorzBreak','InducedVertBreak']]

In [4]:
#pitch type unique values
pitch_type = dft['TaggedPitchType'].unique()
pitch_type

array(['FourSeamFastBall', 'Slider', 'Curveball', 'ChangeUp', 'Splitter',
       'Sinker', 'Fastball', 'TwoSeamFastBall', 'Cutter', 'Undefined',
       'Other', 'Knuckleball', 'OneSeamFastBall'], dtype=object)

In [5]:
#group all fastballs and sinkers together and name them 'Fastball'
dft.loc[dft['TaggedPitchType'].str.contains('Sinker'), 'TaggedPitchType'] = 'Fastball'

dft.loc[dft['TaggedPitchType'].str.contains('FastBall'), 'TaggedPitchType'] = 'Fastball'

dft.loc[dft['TaggedPitchType'].str.contains('FourSeamFastBall'), 'TaggedPitchType'] = 'Fastball'
dft.loc[dft['TaggedPitchType'].str.contains('TwoSeamFastBall'), 'TaggedPitchType'] = 'Fastball'

In [6]:
#create a new column with a value of 1 if the pitch was a whiff, and 0 if it was not
dft['Whiff'] = dft['PitchCall'].apply(lambda x: 1 if x == 'StrikeSwinging' else 0)
#group by pitcher and pitch type, and sum the number of whiffs
whiffs = dft.groupby(['Pitcher','TaggedPitchType'])['Whiff'].sum()
#Count called strikes
called = dft[dft['PitchCall'] == 'StrikeCalled'].groupby(['Pitcher','TaggedPitchType']).size()
#count the total number of pitches thrown
total_pitches = dft.groupby(['Pitcher','TaggedPitchType']).size()
#group by pitcher and pitch type, and count the total number of swings
total_swings = dft[dft['PitchCall'].isin(['StrikeSwinging','FoulBall','InPlay'])].groupby(['Pitcher','TaggedPitchType']).size()
#remove pitchers with less than 50 total swings
total_pitches = total_pitches[total_pitches > 50]
#total_swings = total_swings[total_swings > 20]
#only show fastballs
#total_swings = total_swings[total_swings.index.get_level_values('TaggedPitchType') == 'Slider']

#calculate the whiff percentage
whiff_percentages = (whiffs / total_swings) * 100
whiff_percentages = whiff_percentages.fillna(0)

#calculate the called strike percentage
called_percentages = (called / total_pitches) * 100
called_percentages = called_percentages.fillna(0)


#CSW
csw = (whiffs + called) / total_pitches * 100
csw = csw.fillna(0)






/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_87856/1700099378.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft['Whiff'] = dft['PitchCall'].apply(lambda x: 1 if x == 'StrikeSwinging' else 0)


In [7]:
#make a dataframe with the called percentages
dfc = pd.DataFrame(called_percentages)
#rename the column
dfc.columns = ['CalledStrike%']
#reset the index
dfc = dfc.reset_index()


In [8]:
#make a new dataframe with the whiff percentages as a column
dfw = pd.DataFrame(whiff_percentages)
#reset the index
dfw = dfw.reset_index()
#rename the column
dfw = dfw.rename(columns={0:'Whiff%'})
#sort the values
dfw = dfw.sort_values(by='Whiff%', ascending=False)


In [9]:
#create a new dataframe from DF that find the averages of each pitch type by 'SpinRate', 'RelSpeed','TaggedPitchType','SpinRate','HorzBreak','InducedVertBreak'
dfavg = dft.groupby(['Pitcher','PitcherTeam','TaggedPitchType']).mean()
dfavg = dfavg.reset_index()




In [10]:
#combine the two dataframes
dfw = pd.merge(dfw, dfavg, on=['Pitcher','TaggedPitchType'])
#remove spinrate and whiff
dfw = dfw.drop(['SpinRate','Whiff'], axis=1)

#move whiff percentage to the back
dfw = dfw[['Pitcher','PitcherTeam','TaggedPitchType','RelSpeed','RelHeight','HorzBreak','InducedVertBreak','Whiff%']]

#sort the values



In [11]:
#create new df with total pitches as a column
dfp = pd.DataFrame(total_pitches)
#reset the index
dfp = dfp.reset_index()
#rename the column
dfp = dfp.rename(columns={0:'TotalPitches'})
#sort the values



In [12]:
#combine the two dataframes
f_whiff = pd.merge(dfw, dfp, on=['Pitcher','TaggedPitchType'])


#move whiff percentage to the back
f_whiff = f_whiff[['Pitcher','PitcherTeam','TaggedPitchType','RelSpeed','RelHeight','HorzBreak','InducedVertBreak','Whiff%','TotalPitches']]


In [13]:
#make a new datafram with csw percentages as a column
dfcsw = pd.DataFrame(csw)
#reset the index
dfcsw = dfcsw.reset_index()
#rename the column
dfcsw = dfcsw.rename(columns={0:'CSW%'})


In [14]:
#merge the two dataframes
f_whiff = pd.merge(f_whiff, dfcsw, on=['Pitcher','TaggedPitchType'])


In [15]:
#merge the called strike percentage
final_whiff = pd.merge(f_whiff, dfc, on=['Pitcher','TaggedPitchType'])
#add a column for CSW
#final_whiff['CSW%'] = final_whiff['Whiff%'] + final_whiff['CalledStrike%']
#move called strike percentage to the back

final_whiff = final_whiff[['Pitcher','PitcherTeam','TaggedPitchType','RelSpeed','RelHeight','InducedVertBreak','HorzBreak','Whiff%','CalledStrike%','CSW%','TotalPitches']]
#round the values
final_whiff = final_whiff.round(2)
#create a new column for rank






In [16]:
#sort the values
final_whiff = final_whiff.sort_values(by='CSW%', ascending=False)
#final_whiff.head(20)

In [17]:
#only show hyannis pitchers
final_whiff = final_whiff[final_whiff['PitcherTeam'] == 'HYA_HAR']
#final_whiff.head(20)